In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn
from plotly import graph_objects as go

In [8]:
# 导入数据
df_orig = pd.read_csv('champagne.csv',index_col=0)
predict_field = 'value'
df_orig.columns = [predict_field]
df_orig

,value
Month,
1964-01,2815
1964-02,2672
1964-03,2755
1964-04,2721
1964-05,2946
...,...
1972-05,4618
1972-06,5312
1972-07,4298


In [16]:
def create_dataset(data:pd.DataFrame, time_step:int):
    arr_x, arr_y=[], []
    for i in range(len(data) - time_step - 1):
        x = data.iloc[i: i + time_step,]
        y = data.iloc[i+time_step,]
        arr_x.append(x.values)
        arr_y.append(y.values)
    return np.array(arr_x), np.array(arr_y)

time_step = 8
dataset_x, dataset_y = create_dataset(df_orig, time_step)

train_size = int(len(dataset_x) * 0.7)
train_x = dataset_x[:train_size]
train_y = dataset_y[:train_size]
print(train_x.shape,train_y.shape)

# 将数据改变形状，RNN 读入的数据维度是 (seq_size, batch_size, feature_size)
train_x = train_x.reshape(-1, 1, time_step)
train_y = train_y.reshape(-1, 1, 1)
print(train_x.shape,train_y.shape)

(67, 8, 1) (67, 1)
(67, 1, 8) (67, 1, 1)


In [19]:
class LSTM_Regression(nn.Module):
    """
        使用LSTM进行回归

        参数：
        - input_size: feature size
        - hidden_size: number of hidden units
        - output_size: number of output
        - num_layers: layers of LSTM to stack
    """
    def __init__(self, input_size, hidden_size, output_size=1, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, _x):
        x, _ = self.lstm(_x)  # _x is input, size (seq_len, batch, input_size)
        s, b, h = x.shape  # x is output, size (seq_len, batch, hidden_size)
        x = x.view(s*b, h)
        x = self.fc(x)
        x = x.view(s, b, -1)  # 把形状改回来
        return x

model = LSTM_Regression(time_step, 6, output_size=1, num_layers=2)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [21]:
for i in range(1000):
    out = model(train_x)
    loss = loss_function(out, train_y)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if (i+1) % 100 == 0:
        print('Epoch: {}, Loss:{:.5f}'.format(i+1, loss.item()))


TypeError: 'int' object is not callable